In [54]:
import numpy as np
from scipy.special import comb
from itertools import combinations
from scipy.io import mmread, mminfo
from os import listdir
from os.path import isfile, join
import time
import pickle

### Start of Elimination Ordering.

#### Normalize Stage

In [55]:
'''Normalize Stage'''
'''preliminaries:
- node == vertex ("vertices" for plural)
- nodes' index start at 0
- all graphs and trees are represented in matrix farm
- valency is the sum of the edges' weights of a node, i.e. the sum of the row in a corresponding index, in numpy:
    np.sum(graph[node_idx])
- currently, the matrix is assumed to be symmetric (undirected)
- a fill is only calculated during the elimination, not during the algorithm process to get the elimination ordering,
and a fill is A[i][j] = 1 iff A[i][k] = A[k][i] = A[j][k] = A[k][j] = 1 then A[i][k] = A[k][i] = A[j][k] = A[k][j] = 0
- the diagonal element is always ignored (must be zero'd first if it's nonzero)
- non zeros' values doesn't matter (unweighted), it's always binary (0,1)
'''
'''independent functions:'''

#for transforiming tree matrices to ordered list
def topological_sort_tree(tree_in):
    #look for the "first" node, which is the node with no incoming edges:
    tree = np.copy(tree_in) #copy the tree so the input wont be affected
    size = tree.shape[0]
    S = []
    for i in range(size):
        #need to exclude disconnected nodes by checking row-wise too:
        if np.sum(tree[:,i]) == 0:
            if np.sum(tree[i]) > 0:
                S.append(i)
    #print("S",S)
    
    enque = lambda q, x: q.append(x)
    deque = lambda q: q.pop(0)
    #kahn's algorithm for topological sort (https://en.wikipedia.org/wiki/Topological_sorting):
    #input: tree, first_nodes
    L = []
    while len(S) > 0:
        n = deque(S)
        L.append(n)
        ms = np.where(tree[n] == 1)[0]    #look for set of destination nodesf rom n (neighbours)
        #for each node m with an edge e from n to m:
        for m in ms:
            tree[n][m] = 0 #remove edge e from the graph
            if np.sum(tree[:,m]) == 0: #if m has no other incoming edges then
                enque(S, m) #insert m into S
    #there should be a final check whether the graph still has some edges, but it isnt necessary for tree cases since trees wont have DAG
    return L

#Breadth-First-Search traversal algorithm:
def BFS(graph, source):
    n_init = graph.shape[0]
    q = []
    enque = lambda q, x: q.append(x)
    deque = lambda q: q.pop(0)
    visited = np.array([0]*n_init)
    distances = np.array([0]*n_init)
    visited[source] = 1
    enque(q, source)
    q_counter = 1 #to keep track how many neighbours enqueued
    path = []
    while q:
        v = deque(q)
        q_counter = q_counter - 1
        neighbours = np.where(graph[v] == 1)[0] #enque all v's neighbours (gamma(v))
        for node_i in neighbours:
            if (visited[node_i] == 0) and (node_i not in q):
                enque(q, node_i)
                visited[node_i] = 1
                q_counter += 1
        #print(v, neighbours, q, q_counter, depth)
        path.append(v)
    return path

#get ordered list from merge forest
def get_ordered_list_merged_vertex(tree, placed_vertex):
    '''algorithm for tree-tracing that covers all scenarios:
    0. transpose the tree (to get the reverse order), due to the nature of the merge procedure, the leaves will be the roots
    1. topological sort to get the root(s)
    2. determine the roots by checking the connections between vertices
    3. if there are more than one roots:
        BFS traverse starting from the placed node to get the ordered lists
    else:
        just use the list from the topological sort as the ordered list
    '''
    #print("edges:", np.where(tree.T == 1))
    topological_list = topological_sort_tree(tree.T)
    #print("topological_list",topological_list)
    '''
    print("topological_list",topological_list)
    print(tree)
    print(tree.T)
    '''
    #check the number of roots and get the corresponding roots:
    length = len(topological_list)
    roots = [topological_list[0]]
    for i_elem in topological_list:
        non_root_found = False
        for j_elem in topological_list:
            if i_elem != j_elem:
                if tree.T[i_elem][j_elem] == 0:
                    #print(i_elem, j_elem)
                    roots.append(j_elem)
                else:
                    non_root_found = True
                    break
        if non_root_found:
            break
    #print("roots",roots)
    #if more than one roots, do BFS starting from the placed node, else just use the topological_list:
    ordered_list = None
    #print("ordlist bfs reversed:",list(reversed(BFS(tree, placed_vertex))))
    if len(roots) > 1:
        #ordered_list = BFS(tree.T, placed_vertex)
        ordered_list = list(reversed(BFS(tree, placed_vertex)))
        #print("orderedlist bfs",ordered_list)
    else:
        ordered_list = topological_list
        #print("orderedlist",topological_list)
    #print("ordered_list",ordered_list)
    return ordered_list

#clique checker:
def clique_check(graph, vertices_idx):
    #get subgraph, by slicing indexes:
    subgraph = graph[vertices_idx][:,vertices_idx]
    n = subgraph.shape[0]
    #check for clique from subgraph:
    upper_tri = subgraph[np.triu_indices(n, 1)]
    return np.sum(upper_tri) == comb(n, 2)

#subset checker:
def check_subset(graph, neighbours):
    bool_subset = False
    j_get = None
    for j_node in neighbours:
        #probably need to be stopped earlier? instead of taking the last neighbour index
        gamma_j = np.where(graph[j_node] == 1)[0]
        j_T = np.append(gamma_j, j_node) #j^up_tack = j union gamma(j):= j added to its neighbours
        if set(neighbours).issubset(set(j_T)): #gamma(i) \subset j^up_tack
            bool_subset = True
            j_get = j_node
            break #stop when found
    return bool_subset, j_get

#more accurate way of checking the total nodes within a graph, since the edge is represented 
#by the value of A[i][j] cell, e.g if i <-> j is connected, it means A[i][j] = A[j][i] 1, otherwise 0, 
#so the size of the matrix may not correspond to the total number of nodes
def get_total_nodes(graph, row_size):
    counter = 0
    for i in range(row_size):
        if np.sum(graph[i]) > 0:
            counter += 1
    return counter

'''end'''


def initialize(graph):
    '''function for data initialization, returns:
    - e vector placeholder
    - weight vector w
    - empty merge forest
    - first zero and last zero idxes
    - deleted bool array
    '''
    n = graph.shape[0]
    e = np.array([-1]*n) #for now the placeholder is an array of -1
    w = np.array([1]*n) #weight vector for merge forest
    merge_forest = np.zeros((n,n)) #merge forest for assessment criteria
    deleted = np.array([False]*n)
    first_zero = 0; last_zero = -1
    return e, w, first_zero, last_zero, merge_forest, deleted

#normalize stage:
def normalize(graph):
    global deleted, e, w, first_zero, last_zero, merge_forest
    n = n_init = graph.shape[0] #number of nodes
    '''e = np.array([-1]*n) #for now the placeholder is an array of -1
    w = np.array([1]*n) #weight vector for merge forest
    merge_forest = np.zeros((n,n)) #merge forest for assessment criteria'''
    modified = np.array([1]*n) #modified = 1, otherwise 0'''


    #normalize stage
    #for now, cyclic ordering assumption: start from the 1st index to last idx, hence for-loop
    #need merge check for every node passed, by w[i] > 1
    #print("i, n, m, valency, e, summodified, firstzero, lastzero")
    while np.sum(modified) > 0:
        #print()
        #for i in range(n_init):
        for i in range(n_init):
            #check if it is already deleted, if yes, skip:
            if deleted[i]: #deleted in prev round
                modified[i] = 0 #set modified to 0
                #print("already deleted:",i)
                continue
            if np.sum(modified) == 0:
                break
            #recalculate all of the values:
            n = get_total_nodes(graph, n_init) #recalculate n by excluding zero vectored rows (disconnected vertices)
            valencies = np.array([np.sum(graph[j]) for j in range(n_init)]) #needs to recalculate the valency for each update due to the graph-change
            #print(i,n,m,valency,valencies,e,modified)
            mean_valency = np.sum(valencies)/n #get mean valency
            max_valency = np.max(valencies) #get max valency
            valency = np.sum(graph[i]) #get vertex's valency
            m = np.min([mean_valency, np.floor(n**(1/4) + 3)])
            #m = np.floor(n**(1/4) + 3) #probably this is the correct interpretiation
            #print("mean_valency, np.floor(n**(1/4) + 3)",mean_valency, np.floor(n**(1/4) + 3))
            neighbours = np.where(graph[i] == 1)[0] #get the neighbours of i
            #print(i,n,m,valency,e,np.sum(modified),first_zero, last_zero)
            #print("neighbours",neighbours)
            #check all of the conditions based on the valency
            if valency == n-1:
                ##always check for merge - i.e w[i] > 1
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list #lastzero placement
                    last_zero -= len_ord_list #decrement last zero by the size of the ordered list 
                else:
                    #add to the last zero and decrement the indexer:
                    e[last_zero] = i
                    last_zero -= 1
                graph[i] = graph[:,i] = 0  #remove from graph by deleting edges
                deleted[i] = True
                #graph = np.delete(graph, i, 0) #delete from graph -- this should be the proper deletion method, though not sure if it's faster
                #graph = np.delete(graph, i, 1)
                modified[neighbours] = 1 #set neighbours as modified
                #print("rule 1")
            elif (valency > np.ceil(n/2)) and (valency == max_valency):
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list
                    last_zero -= len_ord_list
                else:
                    e[last_zero] = i
                    last_zero -= 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                #print("rule 2")
            elif valency <= 1:
                #e.insert(0, i) #place vertex first
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list #increment the first zero by the size of the ordered list
                else:
                    #add to the first zero pos and increment the indexer:
                    e[first_zero] = i
                    first_zero += 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                #print("rule 3")
            elif valency == 2:
                #e.insert(0, i)
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list
                else:
                    #add to the first zero pos and increment the indexer:
                    e[first_zero] = i
                    first_zero += 1
                graph[neighbours[0]][neighbours[1]] = graph[neighbours[1]][neighbours[0]] = 1 #make edge between them -- fill the value of the cell with 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                #print("rule 4")
            elif (valency <= m) and (clique_check(graph, neighbours)):
                #e.insert(0, i)
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    #print("tree[i]",np.where(merge_forest[i] == 1))
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list
                    #print("place multiple nodes",ordered_list)
                else:
                    #add to the first zero pos and increment the indexer:
                    #print("place one node")
                    e[first_zero] = i
                    first_zero += 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                #print("rule 5")
            elif (valency <= m): 
                bool_subset, j_node = check_subset(graph, neighbours) #gamma(i) \subset j^uptack, j \in gamma(i)
                if bool_subset:
                    merge_forest[j_node][i] = 1 #merge i into j, add directed edge j->i
                    w[j_node] += 1 #increment weight
                    graph[i] = graph[:,i] = 0
                    deleted[i] = True
                    modified[neighbours] = 1
                    #print("rule 6")
                    #print(neighbours, modified[neighbours])
                    #print("merged",i,j_node)
            modified[i] = 0 #set vertex as unmodified
            #print("w,deleted",w[i],np.where(deleted == True)[0])
            #print()
    #return e, w, first_zero, last_zero, merge_forest
    #return first_zero, last_zero

In [38]:
'''merge-forest placement test'''
dummy_order = np.array([7,9,8,-1,-1,-1,-1,-1,5,6])
dummy_placed = 3
dum_fz = 3
dum_lz = -3

#topological sort:
t = np.array([[0,0,1,0,0], #0-2,1-3,4-2
              [0,0,0,1,0],
              [0,0,0,0,0],
              [0,0,0,0,0],
              [0,0,1,0,0]
             ])
'''
t = np.array([[0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [0,0,0,0]])
'''


ordered_list = get_ordered_list_merged_vertex(t, dummy_placed)
print(dummy_order,ordered_list)
#place first:
length = len(ordered_list)
length_do = len(dummy_order)
print(length_do, length)
dummy_order[dum_fz : dum_fz + length] = ordered_list #firstzero placement
print(length_do + dum_lz - length, length_do + dum_lz + 1) #5,8
dummy_order[length_do + dum_lz - length + 1: length_do + dum_lz + 1] = ordered_list #lastzero placement
print(dummy_order)

S [2, 3]
[ 7  9  8 -1 -1 -1 -1 -1  5  6] [3, 1]
10 2
5 8
[ 7  9  8 -1 -1 -1  3  1  5  6]


In [3]:
'''graphs collection for normalization testing'''
#test array/graph from https://people.sc.fsu.edu/~jburkardt/m_src/rcm/rcm.html
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
print(graph)

#nauru graph
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))

'''testing-ground for normalize stage'''
e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph)
print(normalize(graph))
print("e, w, first_zero, last_zero, merge_forest, deleted",e, w, first_zero, last_zero, merge_forest, deleted)

e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph_1)
print(normalize(graph_1))
print("e, w, first_zero, last_zero, merge_forest, deleted",e, w, first_zero, last_zero, merge_forest, deleted)


[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]
True
i, n, m, valency, valencies, e, modified, firstzero, lastzero

0 9 2.6666666666666665 3 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1 1 1] 9 0 -1
neighbours [2 7 8]
w,deleted [1 1 1 1 1 1 1 1 1] []

1 9 2.6666666666666665 4 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1 1 1] 8 0 -1
neighbours [2 5 6 7]
w,deleted [1 1 1 1 1 1 1 1 1] []

2 9 2.6666666666666665 2 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1 1 1] 7 0 -1
neighbours [0 1]
rule 4
w,deleted [1 1 1 1 1 1 1 1 1] [2]

3 8 2.75 2 [3 4 0 2 2 2 3 4 2] [ 2 -1 -1 -1 -1 -1 -1 -1 -1] [1 1 0 1 1 1 1 1 1] 8 1 -1
neighbours [7 8]
rule 4
w,deleted [1 1 1 1 1 1 1 1 1] [2 3]

4 7 2.857142857142857 2 [3 4 0 0 2 2 3 4 2] [ 2  3 -1 -1 -1 -1 -1 -1 -1] [1 1 0 0 1 1 1 1 1] 7 2 -1
neighbours [6 7]
rule 4


C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in long_scalars


In [108]:
"""testing the merge rule (r6) using banded matrix"""
graph_2 = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
print("g2:")
print("symm", np.allclose(graph_2, graph_2.T, rtol=1e-05, atol=1e-08))
print("clique", clique_check(graph_2, [1,2,3]))
print("1^up_tack", check_subset(graph_2, [1,2,3]))
e, w, first_zero, last_zero, merge_forest = initialize(graph_2)
print(normalize(graph_2, e, w, first_zero, last_zero, merge_forest))


graph_3 = np.array([[0,1,1,1],
                    [1,0,1,0],
                    [1,1,0,1],
                    [1,0,1,0]])
print("g3:")
print("clique", clique_check(graph_3, [1,2,3]))
print("1^up_tack", check_subset(graph_3, [1,2,3]))

#test using another bamded matrix
from scipy.sparse import diags
graph_4 = diags([1,1,1, 1, 0, 1, 1,1,1], [-4,-3,-2,-1, 0, 1,2,3,4], shape=(8, 8), dtype=int).toarray()
print("g4:")
print(graph_4)
print("clique", clique_check(graph_4, [1,2,3]))
print("1^up_tack", check_subset(graph_4, [1,2,3]))
e, w, first_zero, last_zero, merge_forest = initialize(graph_4)
print(normalize(graph_4, e, w, first_zero, last_zero, merge_forest))

g2:
symm True
clique True
1^up_tack (True, 1)
i, n, m, valency, valencies, e, modified, firstzero, lastzero

4.285714285714286 4.0
0 7 4.0 3 [3 4 5 6 5 4 3] [-1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1] 7 0 -1
rule 5
w [1 1 1 1 1 1 1]

4.0 4.0
1 6 4.0 3 [0 3 4 5 5 4 3] [ 0 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1] 6 1 -1
rule 5
w [1 1 1 1 1 1 1]

3.6 4.0
2 5 3.6 3 [0 0 3 4 4 4 3] [ 0  1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1] 5 2 -1
rule 5
w [1 1 1 1 1 1 1]

3.0 4.0
3 4 3.0 3 [0 0 0 3 3 3 3] [ 0  1  2 -1 -1 -1 -1] [0 0 0 1 1 1 1] 4 3 -1
rule 1
w [1 1 1 1 1 1 1]

2.0 4.0
4 3 2.0 2 [0 0 0 0 2 2 2] [ 0  1  2 -1 -1 -1  3] [0 0 0 0 1 1 1] 3 3 -2
rule 1
w [1 1 1 1 1 1 1]

1.0 4.0
5 2 1.0 1 [0 0 0 0 0 1 1] [ 0  1  2 -1 -1  4  3] [0 0 0 0 0 1 1] 2 3 -3
rule 1
w [1 1 1 1 1 1 1]

nan 3.0
6 0 nan 0 [0 0 0 0 0 0 0] [ 0  1  2 -1  5  4  3] [0 0 0 0 0 0 1] 1 3 -4
rule 3
w [1 1 1 1 1 1 1]

[0 1 2 6 5 4 3]
g3:
clique False
1^up_tack (True, 2)
g4:
[[0 1 1 1 1 0 0 0]
 [1 0 1 1 1 1 0 0]
 [1 1 0 1 1 1 1 0]
 [1 1 1 0 1 1 1 1]
 [

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:189: RuntimeWarning: invalid value encountered in long_scalars


#### Separation Stage

In [56]:
'''dijkstra https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm''' 
#function to help djikstra algorithm:
def get_min_distance_vertex(Q, distances):
    min_dist = float("inf")
    min_v = None
    for v in range(Q.shape[0]):
        if (distances[v] < min_dist) and (Q[v] == 1):
            min_dist = distances[v]
            min_v = v
    return min_dist, min_v

#start of dijkstra algorithm:
def dijkstra_shortest_path(graph, source):
    #n_init = get_total_nodes(graph, graph.shape[0])
    n_init = graph.shape[0]
    Q = np.array([1]*n_init)
    #print(Q)
    #source = root = 0
    distances = np.array([float("inf")]*n_init) #set distance vector
    distances[source] = 0
    prev = np.array([None]*n_init)

    while np.sum(Q) > 0:
        _, u = get_min_distance_vertex(Q, distances) #get the vertex with minimum distance
        Q[u] = False #remove u from Q
        neighbours = np.where(graph[u] == 1)[0]
#        print("len(Q), neighbours",len(Q), neighbours)
        for v in neighbours:
            if Q[v] == 1:
                alt = distances[u] + graph[u][v] #distance is equal to the weight of the edge between u and v
                if alt < distances[v]:
                    distances[v] = alt
                    prev[v] = u
                #print(alt)
    return distances, prev

#function to find max valency from nodes
def get_max_valency(subset_nodes, valencies):
    max_valency = -float("inf")
    max_vertex = None
    for m in subset_nodes:
        if valencies[m] > max_valency:
            max_valency = valencies[m]
            max_vertex = m
    return max_vertex, max_valency

'''Separate Stage'''
def separate(graph):
    global deleted, e, w, first_zero, last_zero, merge_forest
    n_init = graph.shape[0] #actual graph size
    
    '''RCM part'''
    #1, d=0, pick vertex e with max valency:
    #print("#1: ")
    d_prime = 0
    n_nodes = get_total_nodes(graph, graph.shape[0]) #current total nodes
    valencies = np.array([np.sum(graph[i]) for i in range(n_init)])
    e_sep = np.argmax(valencies) #get the node with max valency

    #2, need to find a set of M with max distansce from e, which requires BFS or djikstra:
    #print("#2: ")
    distances, _ = dijkstra_shortest_path(graph, e_sep)
    #print("distances",distances)
    conn_components = np.where(distances != np.inf)[0] #indexes of connected components within the subgraph where e resides
    conn_distances = distances[conn_components] #distances of connected components (distances excluding inf)
    s = conn_components.shape[0] #total connected components
    d = np.max(conn_distances) #max distance from e
    M = np.where(distances == d)[0] #set of vertices with max distance from e
    #print("n_init, valencies, e_sep, s, d, M, conn_distances")
    #print(n_init, valencies, e_sep, s, d, M, conn_distances)
    
    
    #3, if d'>d, d'=d, pick a vertex from M with max valency, back to 2 if the first e is close to the second e
    #print("#3: ")
    loopcount = 0 #for repetition statistics
    while d>d_prime:
        #print("d, d_prime, e_sep",d, d_prime, e_sep)
        d_prime = d
        max_vertex,_ = get_max_valency(M, valencies)
        #print("M, valencies",M, valencies)
        e_sep = max_vertex

        #do 2 again:
        distances, _ = dijkstra_shortest_path(graph, e_sep)
        conn_components = np.where(distances != np.inf)[0] #indexes of connected components within the subgraph where e resides
        conn_distances = distances[conn_components] #distances of connected components (distances excluding inf)
        s = conn_components.shape[0] #total connected components
        d = np.max(conn_distances) #max distance from e
        M = np.where(distances == d)[0] #set of vertices with max distance from e
        loopcount+=1
    #print("RCM loopcount", loopcount)
    #print("n_init, valencies, e_sep, s, d, M, conn_distances")
    #print(n_init, valencies, e_sep, s, d, M, conn_distances)
    '''end of RCM'''

    
    #3.5, get the n_k from e, 0<=k<=d, d=max distance, k \in Z
    #print("#3.5: n_k from e, 0<=k<=d, d=max distance")
    max_d = np.max(conn_distances).astype(int)
    n_arr = np.zeros(max_d+1)
    for i in range(0,max_d+1):
        n_arr[i] = np.where(conn_distances == i)[0].shape[0]
    #print("n_arr",n_arr)
    
    
    #4, initialization of several variables:
    ##NOTE: there are two n's, n_k and n_{k+1}, which will be used for comparison in a condition.
    #print("#4: ")
    k=0; N=[np.array([e_sep])]; n=[1]; u = s-1; tried = np.array([0]*n_init); tried[e_sep] = 1
    
    seploop = 0
    while True:
        #first line:
        #gamma_{k+1}(e):=get neighbours/set of points from e with the distance of k+1
        N_next = np.where(distances == k+1)[0] #get the set of neighbours with distance = k+1
        N.append(N_next)
        n.append(len(N[k+1])) #or sum of weights?
        u -= n[k+1]
        #print("k,N,n,u",k,N,n,u)

        #print("n_arr[k] <= n_arr[k+1] < n_arr[k+2]",n_arr[k] <= n_arr[k+1] < n_arr[k+2])
        
        #print("k+2, len(n_arr), d",k+2, len(n_arr), d)
        '''if k+2 < len(n_arr): #temporary fix, by skipping the block if k+2 >= len(n)
            if (n_arr[k] <= n_arr[k+1] < n_arr[k+2]):
                k += 1
                continue'''
        if (k < d-1) and (n_arr[k] <= n_arr[k+1] < n_arr[k+2]) and (u > 0.4*s): #another fix, by adding more skip-conditions
            #print("(k < d-1) and (n_arr[k] <= n_arr[k+1] < n_arr[k+2]) and (u > 0.4*s) condition reached")
            k += 1
            continue
        
        #second line, determining "in degrees":
        #c = {} #need to know which c corresponds to which node, so probably use a dictionary
        c = np.zeros(n_init)
        j_idxs = N[k+1] #to keep track the used node indexes
        for node_j in N[k+1]: #indexing not by nodes' indices, but by c's internal index
            gamma_j = np.where(graph[node_j] == 1)[0]
            c[node_j] = (np.intersect1d(gamma_j, N[k])).shape[0]
            #print("gamma_j, N[k]",gamma_j, N[k])
        #print("c[j_idxs]",c[j_idxs])

        #third line, determining the "out-weights" (weights from normalization stage):
        #b = {} #same reason with c
        b = np.zeros(n_init)
        i_idxs = N[k]
        for node_i in N[k]:
            gamma_i = np.where(graph[node_i] == 1)[0]
            out_w_nodes = np.intersect1d(gamma_i, N[k+1])
            b[node_i] = np.sum(w[out_w_nodes]) #w = weights from normalization, need to know which value belongs to which
            #print("gamma_i, N[k+1]",gamma_i, N[k+1])
        #print("b",b)
        
        
        #fourth line:
        while n[k] > 0:
            #print("n[k]>0",n[k] > 0)
            if (u > 0.4*s) and (n[k+1] < n[k]): #threshold = 0.4s
                #print("(u > 0.4*s) and (n[k+1] < n[k])",(u > 0.4*s) and (n[k+1] < n[k]))
                break
            #place i with largest b_i last: (the rule should follow the placement rule in normalization)
            #new condition to check, when b_i = 0, then break:
            if np.sum(b) == 0:
                #print("b_i all zero")
                #print("k,d,b,c",k,d,b,c)
                break
            #
            placed = np.argmax(b)
            #print("placed",placed)
            ##start of temporary fix
            #if b[placed] > 0: #meaning, gamma(i) \intersect N_{k+1} is not {}
            if w[placed] > 1:
                ordered_list = get_ordered_list_merged_vertex(merge_forest, placed)
                len_e = len(e)
                len_ord_list = len(ordered_list)
                e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list
                last_zero -= len_ord_list
            else:
                e[last_zero] = placed
                last_zero -= 1
            graph[placed] = graph[:,placed] = 0
            deleted[placed] = True
            b[placed] = 0 #remove from b
            #print("e,fz,lz after placement:",e,first_zero,last_zero)
            #decrement s, n_k, c_j:
            #print("s,n[k],c",s,n[k],c)
            s -= 1; n[k] -= 1; c[N[k+1]] -= 1
            ##end of temporary fix#
            #print("s,n[k],c",s,n[k],c)
            #if c_j == 0: ......
            #drop c_j from N; incr u; decr n[k+1]:
            for node_j in N[k+1]:
                if c[node_j] == 0:
                    N[k+1] = N[k+1][N[k+1] != node_j] #drop cj from N
                    u += 1; n[k+1] -= 1
                    #print("N, u, n, c[node_j]",N, u, n, c[node_j])
        if n[k] == 0: ##NOTE: this part is a little bit uncanny, since in first iter the n[k] will always reach 0
            break
        tried[N[k]] = 1; k += 1 #mark all i \in N_k as tried, increment k
        seploop+=1
        #print("\n seploop",seploop)

    #print(e)
    #return graph, e, w, first_zero, last_zero, merge_forest
    return first_zero, last_zero
            #break #for loop breaking purpose during tests -- removed on actual scenario
        #break #for loop breaking purpose during tests -- removed on actual scenario

'''dummy data'''
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])

#nauru graph (bipartite)
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
#print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))
graph = graph_1

'''
graph = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
'''

'''n_init = graph.shape[0]
w = np.array([1]*n_init)
first_zero = 0; last_zero = -1
e = np.zeros(n_init)'''
e,w,first_zero, last_zero, merge_forest, deleted = initialize(graph) 
'''end of dummy data'''

separate(graph)
print("e,w,first_zero, last_zero, merge_forest, deleted: \n",e,w,first_zero, last_zero, merge_forest, np.where(deleted == True))

e,w,first_zero, last_zero, merge_forest, deleted: 
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 18 10  1  3] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 0 -5 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

### Combining both normalize and separate stage

In [57]:
'''input data: '''
'''graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])'''
#nauru graph (bipartite)
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
#print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))

#custom graph
graph_2 = np.array([
    [0,1,0,0,1,0,1,1],
    [1,0,0,0,1,0,0,1],
    [0,0,0,0,0,0,0,1],
    [0,0,0,0,1,1,1,1],
    [1,1,0,1,0,1,1,1],
    [0,0,0,1,1,0,1,0],
    [1,0,0,1,1,1,0,1],
    [1,1,1,1,1,0,1,0]
])

def elimination_ordering(graph, log=False):
    #alternate normalize and separate while the graph is not empty
    i=0
    while np.sum(graph) > 0:
        if np.sum(graph) == 0:
            break
        if log:
            print("Normalize:")
        normalize(graph)
        if log:
            print("e, w, first_zero, last_zero, deleted", e, w, first_zero, last_zero, np.where(deleted == True))
        if np.sum(graph) == 0:
            break
        if log:
            print("\n Separate:")
        separate(graph)
        if log:
            print("e, w, first_zero, last_zero, deleted \n", e, w, first_zero, last_zero, np.where(deleted == True))
        #print(graph, merge_forest)
        if log:
            print("==================NEW ROUND======================= \n")
        print("stage iteration:",i)
        i += 1
    return e

'''initialization'''
graph = graph_2
graph_elim = np.copy(graph)
graph_elim2 = np.copy(graph_elim)
e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph) #must be on global scope

'''ordered elimination'''
start = time.time()
elimination_ordering(graph)
fills, _ = eliminate(graph_elim, e)
print("eli",fills,e)
end = time.time()
print("time elapsed for elimination ordering: ",end - start,"s")


'''test using metis'''
#adj_mat_to_metis_file(graph_elim2, "custgraph.metisgraph")
#metis_order = iperm_to_orderlist("matrices/nauru_bipartite.metisgraph.iperm")
metis_order = iperm_to_orderlist("matrices/custgraph.metisgraph.iperm")
fills, _ = eliminate(graph_elim2, metis_order)
print("metis", fills, metis_order)

eli 1 [1 2 5 7 6 0 4 3]
time elapsed for elimination ordering:  0.008007526397705078 s
metis 0 [2 1 0 5 3 7 4 6]


C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in long_scalars


### Utilities

In [153]:
def eliminate(graph, elimination_order):
    '''elimination function: eliminates the vertices based on the order resulted from elimination ordering algorithms
    - takes in the vertices order from the any elimination ordering algorithms (e.g. METIS' nested dissection)
    - fill will be added when the "center" of the vertex is eliminated, e.g., 1-2-3, eliminate 2, fill(1-3), fill_count+=1
    - for now, assume the fill will be comb(n,2), so if there are 3 vertices which depend on an eliminated vertex, there will be 6 fills
    '''
    count_fill = 0
    for node_i in elimination_order:
        #find neighbours and fill the fill-in indexes:
        neighbours = np.where(graph[node_i] == 1)[0]
        fill_idxs = list(combinations(neighbours, 2))
        #fill-in the edge of i's neighbours:
        if len(fill_idxs) > 0:
            #check if the edges are present, if there are no edges, add them:
            for fill in fill_idxs:
                if graph[fill[0]][fill[1]] == 0:
                    graph[fill[0]][fill[1]] = graph[fill[1]][fill[0]] = 1
                    count_fill += 1
        #remove the edges of i:
        graph[node_i] = graph[:,node_i] = 0
    return count_fill, graph

def adj_mat_to_metis_file(graph, filename):
    '''write adjacency matrix to file'''
    first_line = np.array([graph.shape[0], int(np.sum(graph)/2)]) #[nodes, edges]
    adj_list = []
    for i in range(graph.shape[0]):
        neighbours = np.where(graph[i] == 1)[0]
        neighbours += 1
        adj_list.append(neighbours)
    adj_list = np.array(adj_list)
    
    with open(filename,"w") as f:
        f.write(str(first_line[0])+" "+str(first_line[1])+"\n")
        for i in range(adj_list.shape[0]):
            for j in range(adj_list[i].shape[0]):
                f.write(str(adj_list[i][j])+" ")
            f.write("\n")
    print("writing",filename,"done!")

def iperm_to_orderlist(filename):
    '''read iperm from ndmetis and convert it to list'''
    f = open(filename, "r")
    order = []
    for x in f:
        order.append(int(x))
    order = np.array(order)
    #according to metis documentatoin:
    actual_order = np.zeros(order.shape[0])
    for i in range(order.shape[0]):
        actual_order[i] = np.where(order == i)[0]
    actual_order = actual_order.astype(np.int64, copy=False)
    return actual_order

def load_matrix_market(filename, get_mat_meta=False):
    '''test using matrices from matrix market'''
    #filename = "matrices/bcsstm01.mtx.gz"
    metadata = mminfo(filename)
    Matrix = mmread(filename)
    A = Matrix.toarray()
    #print(A)
    #print(np.nonzero(A))
    '''preprocess the matrix'''
    A = A.astype(np.int64, copy=False)
    '''    
    #symmetrize the matrix:
    A = A + A.T
    #print("symmetrize:")
    #print(A)
    #set diagonals to zero:
    np.fill_diagonal(A, 0)
    #print("diag")
    #print(A)
    #if a nonzero element is >0 or <0, set it to 1:
    #print("nz")
    A[np.nonzero(A)] = 1
    #print(A)
    '''
    if get_mat_meta:
        return A, metadata
    else:
        return A

def bipartization(G):
    '''square -> bipartite'''
    A = np.copy(G)
    len_A = A.shape[0]
    B = np.zeros([len_A*2, len_A*2])
    len_B = B.shape[0]
    B[:len_A,len_A:len_B] = A #top right
    B.T[:len_A,len_A:len_B] = A #bottome left
    B[np.nonzero(B)] = 1
    return B    
    
def symmetrization(G):
    '''square -> symmetric'''
    A = np.copy(G)
    #symmetrize the matrix:
    A = A + A.T
    #set diagonals to zero:
    np.fill_diagonal(A, 0)
    #if a nonzero element is >0 or <0, set it to 1:
    #print("nz")
    A[np.nonzero(A)] = 1
    return A

def grid_generator(p, q, k, p_dep=0, q_dep=0):
    '''p*q grid generator, p = row, q = col
    if p_dep = 1: p=q,
    elif p_dep = 2: p=q**2
    else p is an input parameter
    if q_dep = 1: q = 2**k
    else q is an input parameter
    '''
    if q_dep == 1:
        q = 2**k
    if p_dep == 1:
        p = q
    elif p_dep == 2:
        p = q**2
    print('p,q,k',p,q,k)
    grid = np.zeros((p*q, p*q)) #grid matrix
    diag = np.ones(q)
    sub_grid = np.zeros((q,q))
    np.fill_diagonal(grid[q:], diag) #lower diagonal main grid
    np.fill_diagonal(grid[:,q:], diag) #upper diagonal main grid
    np.fill_diagonal(sub_grid[1:], diag) #lower diagonal for subgrid
    np.fill_diagonal(sub_grid[:,1:], diag) #upper diagonal for subgrid
    for i in range(p):
        grid[i*q:(i*q)+q, i*q:(i*q)+q] = sub_grid
    return grid

graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
order = [2, 3, 4, 5, 6, 8, 1, 0, 7] #from the elimination ordering result
countfill,_ = eliminate(graph, order)
print("total fill:" ,countfill)

#banded matrix 2m+1 = 15
graph_2 = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
order = [0, 1, 2, 6, 5, 4, 3]
countfill,_ = eliminate(graph, order)
print("total fill:" ,countfill)

print(iperm_to_orderlist("matrices/disconn_graph.metisgraph.iperm"))

A = load_matrix_market("matrices/bcsstk01.mtx.gz")
print(np.nonzero(A))

total fill: 3
total fill: 0
[7 1 3 0 2 6 5 4]


FileNotFoundError: [Errno 2] No such file or directory: 'matrices/bcsstk01.mtx.gz'

### Testing schemes

In [62]:
'''matrices pre-processing'''
mypath = "matrices/mm/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
#for all matrices, load:
for f in onlyfiles:
    G_mm = load_matrix_market("matrices/mm/"+f)
    G_sym = symmetrization(G_mm)
    G_bip = bipartization(G_mm)

    #save all matrices to metis format:
    adj_mat_to_metis_file(G_sym, "matrices/ndmetis_input/"+f+".sym.metisgraph")
    adj_mat_to_metis_file(G_bip, "matrices/ndmetis_input/"+f+".bip.metisgraph")
    
    #run the metis using all the matrices elsewhere.....



['bcsstk01.mtx.gz', 'bcsstm13.mtx.gz', 'ck104.mtx.gz', 'gr_30_30.mtx.gz', 'mcca.mtx.gz', 'mcfe.mtx.gz', 'nnc1374.mtx.gz', 'will57.mtx.gz', 'zenios.mtx.gz']
writing matrices/ndmetis_input/bcsstk01.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/bcsstk01.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/bcsstm13.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/bcsstm13.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/ck104.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/ck104.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/gr_30_30.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/gr_30_30.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/mcca.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/mcca.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/mcfe.mtx.gz.sym.metisgraph done!
writing matrices/ndmetis_input/mcfe.mtx.gz.bip.metisgraph done!
writing matrices/ndmetis_input/nnc1374.mtx.gz.sym.

In [118]:
'''testing main for matrix market'''
#for all matrices, reload:
mm_path = "matrices/mm/"
mmfiles = np.array([f for f in listdir(mm_path) if isfile(join(mm_path, f))])
#onlyfiles = onlyfiles[[0,2]] #only for testing subslices
print(mmfiles)
combinedfiles = np.concatenate((np.array([mmfiles]).T, np.array([mmfiles]).T), axis=1).flatten()
print(combinedfiles)

#requires iperm info to load the mm in order
iperm_path = "matrices/ndmetis_iperm/"
ipermfiles = np.array([f for f in listdir(iperm_path) if isfile(join(iperm_path, f))])
substring_idx = -21 #backward index to ".gz."
print(ipermfiles)

nv = []
ne = []
mat_types = [] #bipartite or not, 1/0
times = []
fills_eli = []
fills_metis = []
fills_eli_ratio = []
fills_metis_ratio = []
eli_metis_ratios = []
#mat_metadata = []

for f in ipermfiles:
    G_mm = load_matrix_market(mm_path+f[:substring_idx])
    G = None
    if ".sym." in f: #even fo symmetric, otherwise bipartite
        G = symmetrization(G_mm); mat_types.append(False)
    elif ".bip." in f:
        G = bipartization(G_mm); mat_types.append(True)
    #mat_metadata.append(metadata) #matrix metadata
    vertices = G.shape[0]
    edges = np.sum(G[np.triu_indices(G.shape[0], 1)]) #sum of upper triangular
    nv.append(vertices); ne.append(edges)
    #initialize: MUST always be in global scope:
    G1 = np.copy(G); G2 = np.copy(G)
    e, w, first_zero, last_zero, merge_forest, deleted = initialize(G)

    #elimination ordering:
    start = time.time()
    elimination_ordering(G)
    fill_eli, _ = eliminate(G1, e)
    fills_eli.append(fill_eli)
    print("eli",fill_eli,e)
    #print("sorted e", sorted(e))
    end = time.time()
    elapsed = end-start
    times.append(float('{0:.3g}'.format(elapsed)))
    print("time elapsed for elimination ordering: ",elapsed,"s")
    fills_eli_ratio.append(float('{0:.3g}'.format(float(fill_eli/edges))))

    #metis:
    metis_order = iperm_to_orderlist(iperm_path+f)
    fill_metis, _ = eliminate(G2, metis_order)
    fills_metis.append(fill_metis)
    print("metis", fill_metis, metis_order)
    fills_metis_ratio.append(float('{0:.3g}'.format(float(fill_metis/edges))))
    #print("sorted e", sorted(metis_order))
    #fills ratio between eli and metis:
    if (fill_eli != 0) and (fill_metis != 0): 
        eli_metis_ratios.append(float('{0:.3g}'.format(float(fill_eli/fill_metis)))) 
    else:
        eli_metis_ratios.append(0)
    
print("mm",combinedfiles)
#print("metadata",mat_metadata)
print("nv",nv)
print("ne",ne)
print("mat_types",mat_types)
print("eli",fills_eli)
print("eli fills/sum(triu)",fills_eli_ratio)
print("eli_times",times)
print("metis",fills_metis)
print("metis fills/sum(triu)",fills_metis_ratio)
print("eli/metis ratio", eli_metis_ratios)

#store results in file
data = {
    "mm": combinedfiles,
    #"metadata": mat_metadata,
    "nv": nv,
    "ne": ne,
    "bipartite": mat_types,
    "eli_fills": fills_eli,
    "eli_ratio": fills_eli_ratio,
    "eli_times": times,
    "metis_fills": fills_metis,
    "metis_ratio": fills_metis_ratio,
    "eli/metis": eli_metis_ratios
}
with open('data.p', 'wb') as fp:
    pickle.dump(data, fp)    
with open('data.p', 'rb') as fp:
    data = pickle.load(fp)
    print("data = ",data)

'''#check ordering result:
e = np.array(e)
e_sort = np.sort(e[e != -1])
print(e_sort)
'''

['bcsstk01.mtx.gz' 'bcsstm13.mtx.gz' 'ck104.mtx.gz' 'gr_30_30.mtx.gz'
 'mcca.mtx.gz' 'mcfe.mtx.gz' 'nnc1374.mtx.gz' 'will57.mtx.gz'
 'zenios.mtx.gz']
['bcsstk01.mtx.gz' 'bcsstk01.mtx.gz' 'bcsstm13.mtx.gz' 'bcsstm13.mtx.gz'
 'ck104.mtx.gz' 'ck104.mtx.gz' 'gr_30_30.mtx.gz' 'gr_30_30.mtx.gz'
 'mcca.mtx.gz' 'mcca.mtx.gz' 'mcfe.mtx.gz' 'mcfe.mtx.gz' 'nnc1374.mtx.gz'
 'nnc1374.mtx.gz' 'will57.mtx.gz' 'will57.mtx.gz' 'zenios.mtx.gz'
 'zenios.mtx.gz']
['bcsstk01.mtx.gz.bip.metisgraph.iperm'
 'bcsstk01.mtx.gz.sym.metisgraph.iperm'
 'bcsstm13.mtx.gz.bip.metisgraph.iperm'
 'bcsstm13.mtx.gz.sym.metisgraph.iperm'
 'ck104.mtx.gz.bip.metisgraph.iperm' 'ck104.mtx.gz.sym.metisgraph.iperm'
 'gr_30_30.mtx.gz.bip.metisgraph.iperm'
 'gr_30_30.mtx.gz.sym.metisgraph.iperm' 'mcca.mtx.gz.bip.metisgraph.iperm'
 'mcca.mtx.gz.sym.metisgraph.iperm' 'mcfe.mtx.gz.bip.metisgraph.iperm'
 'mcfe.mtx.gz.sym.metisgraph.iperm' 'nnc1374.mtx.gz.bip.metisgraph.iperm'
 'nnc1374.mtx.gz.sym.metisgraph.iperm'
 'will57.mtx.gz.bip.

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in double_scalars


stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
eli 419 [25 29 26 10 22 40  9 38 39 41 43 47  7 21 27 19 42 23 31  2 20  4  6  0
 30 24 28 11  5  3 37 44 33 15 18 17  8 36 32 13 16 46 45 35 34 12  1 14]
time elapsed for elimination ordering:  0.22719883918762207 s
metis 265 [26 25 38  7 27 37 21 33 43  9 45  3 13 31 32 15 39 44 29 28 36 23 10 24
 22  6  5 46 42 18 47  0  4 12 16 30 11  2 41 34 20 14 17  8 40  1 35 19]


C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in longlong_scalars


eli 62 [   0    1    2 ... 4004 4005 3902]
time elapsed for elimination ordering:  239.27054595947266 s
metis 62 [3650 3492 3460 ...  386    4    3]
eli 0 [   0    1    2 ... 2001 2002 1899]
time elapsed for elimination ordering:  37.423030853271484 s
metis 0 [1971 1953 1952 ...  115   13    9]
eli 2 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 

metis 13410 [396 390   6   0 366 360   5   1 394 392 306 300 246 240 186 180 126 120
  66  60   3 395 391  36  30 305 301 245 241 185 181 125 121  33 333 273
 213 153  93 365 361  35  31  65  61  96  95  91  90 336 335 331 330 216
 215 211 210  34   4 364 334 362 332 363 393  63  64  94 303 302 304  92
   2  32  62 123 122 124 156 154 155 271 270 272 151 150 152 274 275 276
 214 182 183 184 212 242 243 244 404 398  14   8 374 368  13   9 402 400
 314 308 254 248 194 188 134 128  74  68  11 403 399  44  38 313 309 253
 249 193 189 133 129  41 341 281 221 161 101 373 369  43  39  73  69 104
 103  99  98 344 343 339 338 224 223 219 218  42  12 372 342 370 340 371
 401  71  72 102 311 310 312 100  10  40  70 131 130 132 164 162 163 279
 278 280 159 158 160 282 283 284 222 190 191 192 220 250 251 252   7 277
 247 127  97 217  67 187  37 397 367 337 307 157 884 870 704 690 854 840
 703 691 882 880 878 876 874 872 794 780 701 699 697 695 693 734 720 883
 871 733 721 853 841 852 842 764 750 85

stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
stage iteration: 13
stage iteration: 14
stage iteration: 15
stage iteration: 16
stage iteration: 17
stage iteration: 18
stage iteration: 19
stage iteration: 20
stage iteration: 21
stage iteration: 22
stage iteration: 23
stage iteration: 24
stage iteration: 25
stage iteration: 26
stage iteration: 27
stage iteration: 28
stage iteration: 29
stage iteration: 30
stage iteration: 31
stage iteration: 32
stage iteration: 33
stage iteration: 34
stage iteration: 35
stage iteration: 36
stage iteration: 37
stage iteration: 38
stage iteration: 39
stage iteration: 40
stage iteration: 41
stage iteration: 42
stage iteration: 43
stage iteration: 44
stage iteration: 45
stage iteration: 46
stage iteration: 47
stage iteration: 48
stage iteration: 49
stage iter

stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
stage iteration: 13
stage iteration: 14
stage iteration: 15
stage iteration: 16
stage iteration: 17
stage iteration: 18
stage iteration: 19
stage iteration: 20
stage iteration: 21
stage iteration: 22
eli 18348 [   4    5    6 ... 1532 1410 1471]
time elapsed for elimination ordering:  797.991512298584 s
metis 5666 [2078 1821 1268 ...  472 1879 1849]
stage iteration: 0
stage iteration: 1
stage iteration: 2
stage iteration: 3
stage iteration: 4
stage iteration: 5
stage iteration: 6
stage iteration: 7
stage iteration: 8
stage iteration: 9
stage iteration: 10
stage iteration: 11
stage iteration: 12
eli 7891 [  4   5   6 ... 187  97 173]
time elapsed for elimination ordering:  105.01848769187927 s
metis 2824 [168 167 155 ... 859 904 857]
stage i

'#check ordering result:\ne = np.array(e)\ne_sort = np.sort(e[e != -1])\nprint(e_sort)\n'

In [121]:
'''above cell must be run first'''
#e_sort = e_sort[2:]
print(e_sort)
#check missing:
def printMissingElements(arr, N): 
    # Initialize diff 
    diff = arr[0] 
    for i in range(N): 
        # Check if diff and arr[i]-i 
        # both are equal or not 
        if(arr[i] - i != diff): 
            # Loop for consecutive 
            # missing elements 
            while(diff < arr[i] - i): 
                print(i + diff, end = " ") 
                diff += 1
printMissingElements(e_sort, e_sort.shape[0])

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]


In [91]:
'''disconnected graph checking, if it is assumed as different \gamma'''
G = np.array([
        [0,0,0,1,1,0,0,0],
        [0,0,1,0,0,0,0,0],
        [0,1,0,0,0,0,0,0],
        [1,0,0,0,1,0,0,0],
        [1,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,1,1],
        [0,0,0,0,0,1,0,1],
        [0,0,0,0,0,1,1,0]
    ])

G = np.array([
        [0,1,0,1,0,0,0,1],
        [1,0,0,1,1,0,0,0],
        [0,0,0,0,0,1,1,0],
        [1,1,0,0,1,0,0,1],
        [0,1,0,1,0,0,0,1],
        [0,0,1,0,0,0,1,0],
        [0,0,1,0,0,1,0,0],
        [1,0,0,1,1,0,0,0]
        
    ])
G_elim = np.copy(G)
visited = np.array([False]*G.shape[0])
traverse_count = 0
sub_Gs = []
for i in range(G.shape[0]):
    if not visited[i]:
        sub_G = BFS(G,i)
        sub_Gs.append(sub_G)
        visited[sub_G] = True
print(sub_Gs)

'''generating subgraphs from a whole matrix graph'''
'''Gs = []
for i in range(len(sub_Gs)):
    #set all indices except i to 0
    sliced = []
    for j in range(len(sub_Gs)):
        if j!=i:
            sliced.append(sub_Gs[j])
    print(sliced)
    sliced = [item for sublist in sliced for item in sublist] #flattened list
    print(sliced)
    sub_graph = np.copy(G)
    sub_graph[sliced] = sub_graph[:,sliced] = 0
    Gs.append(sub_graph)
print(Gs)
for G in Gs:
    G_elim = np.copy(G)
    e,w,first_zero,last_zero,merge_forest,deleted = initialize(G)
    #normalize(G)
    separate(G)
    print("e",e)
    print(eliminate(G_elim,e))
    print()
'''
e,w,first_zero,last_zero,merge_forest,deleted = initialize(G_elim) #always initialize before normalize
normalize(G)
print(e)
elim_order_metis = iperm_to_orderlist("disconn_graph.metisgraph.iperm")
print("from ndmetis",elim_order_metis)
G_elim2 = np.copy(G_elim)
fills, _ = eliminate(G_elim, elim_order_metis) #using metis
print("fills",fills)
fills, _ = eliminate(G_elim2, e) #using eli
print("from eli normalize", e)
print("fills",fills)

[[0, 1, 3, 7, 4], [2, 5, 6]]
i, n, m, valency, valencies, e, modified, firstzero, lastzero

0 8 2.75 3 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1 1] 8 0 -1
neighbours [1 3 7]
w,deleted [1 1 1 1 1 1 1 1] []

1 8 2.75 3 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1 1] 7 0 -1
neighbours [0 3 4]
w,deleted [1 1 1 1 1 1 1 1] []

2 8 2.75 2 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1 1] 6 0 -1
neighbours [5 6]
rule 4
w,deleted [1 1 1 1 1 1 1 1] [2]

3 7 2.5714285714285716 4 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 1 1 1 1 1] 5 1 -1
neighbours [0 1 4 7]
w,deleted [1 1 1 1 1 1 1 1] [2]

4 7 2.5714285714285716 3 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 0 1 1 1 1] 4 1 -1
neighbours [1 3 7]
w,deleted [1 1 1 1 1 1 1 1] [2]

5 7 2.5714285714285716 1 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 0 0 1 1 1] 3 1 -1
neighbours [6]
rule 3
w,deleted [1 1 1 1 1 1 1 1] [2 5]

6 5 3.2 0 [3 3 0 4 3 0 0 3] [ 2  5 -1 -1 -1 -1 -1 -1] [0 0 0 0 0

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in long_scalars


In [29]:
G = np.array([
        [0,1,0,1,0,0,0,1],
        [1,0,0,1,1,0,0,0],
        [0,0,0,0,0,1,1,0],
        [1,1,0,0,1,0,0,1],
        [0,1,0,1,0,0,0,1],
        [0,0,1,0,0,0,1,0],
        [0,0,1,0,0,1,0,0],
        [1,0,0,1,1,0,0,0]
        
    ])
upper_tri = G[np.triu_indices(G.shape[0], 1)]
#upper_tri = np.triu_indices(G.shape[0], 1)
print(np.sum(upper_tri))

11


In [147]:
'''Table visualization'''

import pandas as pd
#pd.options.display.float_format = '{:,.2f}'.format



with open('data.p', 'rb') as fp:
    data = pickle.load(fp)
    print("data = ",data)

    print(len(data["mm"]), len(data["nv"]), len(data["eli/metis"]))
df = pd.DataFrame.from_dict(data)



df.style.format({
    'eli_ratio': '{:,.2f}'.format,
    'eli_times': '{:,.2f}'.format,
    'metis_ratio':'{:,.2f}'.format,
    'eli/metis': '{:,.2f}'.format
    #'B': '{:,.3f}'.format,
})

#df.style.set_precision(2)
#df



data =  {'mm': array(['bcsstk01.mtx.gz', 'bcsstk01.mtx.gz', 'bcsstm13.mtx.gz',
       'bcsstm13.mtx.gz', 'ck104.mtx.gz', 'ck104.mtx.gz',
       'gr_30_30.mtx.gz', 'gr_30_30.mtx.gz', 'mcca.mtx.gz', 'mcca.mtx.gz',
       'mcfe.mtx.gz', 'mcfe.mtx.gz', 'nnc1374.mtx.gz', 'nnc1374.mtx.gz',
       'will57.mtx.gz', 'will57.mtx.gz', 'zenios.mtx.gz', 'zenios.mtx.gz'],
      dtype='<U15'), 'nv': [96, 48, 4006, 2003, 208, 104, 1800, 900, 360, 180, 1530, 765, 2748, 1374, 114, 57, 5746, 2873], 'ne': [400.0, 176, 777.0, 62, 10.0, 4, 7744.0, 3422, 2648.0, 1680, 24344.0, 15352, 4286.0, 2248, 281.0, 127, 4.0, 2], 'bipartite': [True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False], 'eli_fills': [1769, 419, 62, 0, 2, 0, 367747, 23229, 5788, 964, 278468, 71144, 18348, 7891, 497, 24, 0, 0], 'eli_ratio': [4.42, 2.38, 0.0798, 0.0, 0.2, 0.0, 47.5, 6.79, 2.19, 0.574, 11.4, 4.63, 4.28, 3.51, 1.77, 0.189, 0.0, 0.0], 'eli_times': [0.795, 0.227, 239.0, 

,mm,nv,ne,bipartite,eli_fills,eli_ratio,eli_times,metis_fills,metis_ratio,eli/metis
0,bcsstk01.mtx.gz,96,400,True,1769,4.42,0.80,1173,2.93,1.51
1,bcsstk01.mtx.gz,48,176,False,419,2.38,0.23,265,1.51,1.58
2,bcsstm13.mtx.gz,4006,777,True,62,0.08,239.00,62,0.08,1.00
3,bcsstm13.mtx.gz,2003,62,False,0,0.00,37.40,0,0.00,0.00
4,ck104.mtx.gz,208,10,True,2,0.20,0.31,2,0.20,1.00
5,ck104.mtx.gz,104,4,False,0,0.00,0.08,0,0.00,0.00
6,gr_30_30.mtx.gz,1800,7744,True,367747,47.50,"1,910.00",42390,5.47,8.68
7,gr_30_30.mtx.gz,900,3422,False,23229,6.79,7.89,13410,3.92,1.73
8,mcca.mtx.gz,360,2648,True,5788,2.19,14.70,3462,1.31,1.67
9,mcca.mtx.gz,180,1680,False,964,0.57,1.71,334,0.20,2.89


In [163]:


grid = grid_generator(3,5,2,p_dep=1,q_dep=1)
print(grid.shape)
print(grid)

p,q,k 4 4 2
(16, 16)
[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]]
